# Dependence Detection

## Loading data
Make sure that all previous enrolled subjects exist in the dataframe.

In [2]:
import pandas as pd
import numpy as np
import json
df_csv= pd.read_pickle("df_noplus/df5.pkl")
all_subjects=df_csv['COURSEID'].value_counts()
##removing any subject enrolled less than 20 times
#all_subjects=all_subjects[all_subjects>=20]
print df_csv.shape
df_csv=df_csv[df_csv["COURSEID"].isin(all_subjects.index)]
print df_csv.shape

(28272, 199)
(28272, 199)


## Decision Trees Planting
Then it is time to generate forest of decision tres

In [5]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier



def get_importances20(subject,pattern_names):
    ##load data
    dfx=df_csv[df_csv['COURSEID']==subject]
    dfx=dfx.iloc[np.random.permutation(len(dfx))]
    ##convert to np.array
    X=dfx.as_matrix( dfx.columns[4:] )
    y=dfx.as_matrix( ['GRADE'] ).T[0]

    # Build a forest and compute the feature importances
    forest = ExtraTreesClassifier(n_estimators=250,
                                  random_state=0)

    forest.fit(X, y)
    importances = forest.feature_importances_
    #std = np.std([tree.feature_importances_ for tree in forest.estimators_],
    #             axis=0)
    indices = np.argsort(importances)[::-1]
    obj=pattern_names[indices[(importances[indices].cumsum()<0.2)  & (importances[indices] > 0.05) ]]
    import_list = [i for i in obj][:3]
    return (import_list, indices, importances)

# unit test
subject='CS284'
print subject
# Print the feature ranking
print("feature important ranking:")

pattern_names=df_csv.columns[4:]
import_list, indices, importances = get_importances20(subject,pattern_names)
print import_list

for f in range(X.shape[1]):
    print("subject: %s, importance:%.3f, acc_sum:%.3f" % (
            pattern_names[indices[f]], importances[indices[f]], importances[indices].cumsum()[f]))

CS284
feature important ranking:
['SC135', 'MA211', 'CS111']
subject: SC135, importance:0.060, acc_sum:0.060
subject: MA211, importance:0.060, acc_sum:0.120
subject: CS111, importance:0.059, acc_sum:0.179
subject: TU154, importance:0.055, acc_sum:0.234
subject: EL171, importance:0.054, acc_sum:0.288
subject: EL172, importance:0.052, acc_sum:0.340
subject: TU120, importance:0.050, acc_sum:0.390
subject: CS102, importance:0.050, acc_sum:0.440
subject: PY228, importance:0.047, acc_sum:0.487
subject: CS101, importance:0.046, acc_sum:0.533
subject: TU110, importance:0.045, acc_sum:0.578
subject: TH161, importance:0.043, acc_sum:0.621
subject: SC185, importance:0.040, acc_sum:0.661
subject: EL070, importance:0.023, acc_sum:0.684
subject: TU100, importance:0.022, acc_sum:0.707
subject: MA212, importance:0.019, acc_sum:0.726
subject: TU130, importance:0.019, acc_sum:0.745
subject: SW111, importance:0.018, acc_sum:0.763
subject: ST216, importance:0.016, acc_sum:0.780
subject: CS105, importance:

## Writing json for Visualization
We are going to use the <a href='http://bl.ocks.org/mbostock/1044242'>Hierarchical Edge Bundling</a> from https://d3js.org/ for a nice visualization.

In [10]:
##generate importance
def add_dot(x):
    return 'cstu.'+x[:2]+'.'+x
pattern_names=df_csv.columns[4:]
data3=[]
for s in all_subjects.index:
    clist = map( add_dot  , get_importances20(s,pattern_names)[0] )
    obj={'name':add_dot(s), 'imports': clist }
    data3.append(obj)
with open('bundle_files/data3.json','w') as fp:
    json.dump(data3, fp,indent=2, separators=(', ', ': '))

## Removing non-connected subjects
Filtering non-connected subject out from data3 

In [13]:
##load imported list, json data3
import json
with open('bundle_files/data3.json','r') as fp:
    data3=json.load(fp)
##filtering non-connected subject out from data3    
imported_subjects={}
for i in data3:
    for j in i['imports']:
        if j in imported_subjects:
            imported_subjects[j]+=1
        else:
            imported_subjects[j]=0
data4=[]
for i in data3:
    if i['name'] in imported_subjects or i['imports']:
        data4.append(i)
    else: 
        print '***removing {}'.format(i)
import json
with open('bundle_files/data4.json','w') as fp:
    json.dump(data4, fp,indent=2, separators=(', ', ': '))

data4

***removing {u'imports': [], u'name': u'cstu.MA.MA332'}
***removing {u'imports': [], u'name': u'cstu.CS.CS261'}
***removing {u'imports': [], u'name': u'cstu.EL.EL395'}
***removing {u'imports': [], u'name': u'cstu.CS.CS374'}
***removing {u'imports': [], u'name': u'cstu.CS.CS302'}
***removing {u'imports': [], u'name': u'cstu.CS.CS401'}
***removing {u'imports': [], u'name': u'cstu.CS.CS342'}
***removing {u'imports': [], u'name': u'cstu.HO.HO201'}
***removing {u'imports': [], u'name': u'cstu.CS.CS402'}
***removing {u'imports': [], u'name': u'cstu.CS.CS365'}
***removing {u'imports': [], u'name': u'cstu.CS.CS289'}
***removing {u'imports': [], u'name': u'cstu.CS.CS326'}
***removing {u'imports': [], u'name': u'cstu.CS.CS288'}
***removing {u'imports': [], u'name': u'cstu.CS.CS487'}
***removing {u'imports': [], u'name': u'cstu.CS.CS489'}
***removing {u'imports': [], u'name': u'cstu.CS.CS296'}
***removing {u'imports': [], u'name': u'cstu.CS.CS367'}
***removing {u'imports': [], u'name': u'cstu.CS.

[{u'imports': [u'cstu.TU.TU154', u'cstu.SC.SC135'], u'name': u'cstu.MA.MA211'},
 {u'imports': [], u'name': u'cstu.CS.CS102'},
 {u'imports': [], u'name': u'cstu.CS.CS101'},
 {u'imports': [], u'name': u'cstu.TU.TU154'},
 {u'imports': [u'cstu.CS.CS101'], u'name': u'cstu.TH.TH161'},
 {u'imports': [u'cstu.CS.CS102'], u'name': u'cstu.CS.CS111'},
 {u'imports': [u'cstu.CS.CS111', u'cstu.CS.CS102', u'cstu.MA.MA211'],
  u'name': u'cstu.CS.CS213'},
 {u'imports': [u'cstu.TU.TU154'], u'name': u'cstu.EL.EL171'},
 {u'imports': [u'cstu.TU.TU154'], u'name': u'cstu.SC.SC135'},
 {u'imports': [u'cstu.TU.TU154'], u'name': u'cstu.PY.PY228'},
 {u'imports': [u'cstu.EL.EL171'], u'name': u'cstu.EL.EL172'},
 {u'imports': [u'cstu.TU.TU154'], u'name': u'cstu.SC.SC185'},
 {u'imports': [u'cstu.TU.TU154', u'cstu.SC.SC135'], u'name': u'cstu.TU.TU110'},
 {u'imports': [u'cstu.CS.CS111', u'cstu.MA.MA211', u'cstu.TU.TU154'],
  u'name': u'cstu.CS.CS223'},
 {u'imports': [u'cstu.TU.TU154'], u'name': u'cstu.TU.TU120'},
 {u'im

## Curriculum Mapp from the Dependence Detection algorithm
The red lines represent suggested prerequisites.

In [31]:
%%HTML
<iframe width="1200" height="750" src="bundle_files/bundle.html"></iframe>

## Sorting by the Importance Score

In [14]:
import operator
sorted_imsub = sorted(imported_subjects.items(), key=operator.itemgetter(1), reverse=True)
sorted_imsub

[(u'cstu.TU.TU154', 17),
 (u'cstu.MA.MA211', 13),
 (u'cstu.CS.CS102', 12),
 (u'cstu.EL.EL171', 9),
 (u'cstu.CS.CS111', 8),
 (u'cstu.SC.SC135', 6),
 (u'cstu.CS.CS101', 5),
 (u'cstu.TH.TH161', 4),
 (u'cstu.TU.TU110', 3),
 (u'cstu.CS.CS284', 3),
 (u'cstu.EL.EL295', 2),
 (u'cstu.EL.EL172', 2),
 (u'cstu.SC.SC173', 2),
 (u'cstu.TU.TU120', 2),
 (u'cstu.MW.MW313', 1),
 (u'cstu.CS.CS223', 1),
 (u'cstu.CS.CS341', 1),
 (u'cstu.SW.SW111', 1),
 (u'cstu.SC.SC185', 1),
 (u'cstu.CS.CS395', 1),
 (u'cstu.CS.CS214', 1),
 (u'cstu.CS.CS486', 1),
 (u'cstu.CS.CS311', 1),
 (u'cstu.AT.AT326', 1),
 (u'cstu.SW.SW365', 1),
 (u'cstu.MA.MA212', 1),
 (u'cstu.ST.ST216', 1),
 (u'cstu.SC.SC123', 0),
 (u'cstu.CS.CS385', 0),
 (u'cstu.CS.CS222', 0),
 (u'cstu.EG.EG241', 0),
 (u'cstu.MW.MW314', 0),
 (u'cstu.CS.CS301', 0),
 (u'cstu.SW.SW335', 0),
 (u'cstu.CS.CS398', 0),
 (u'cstu.CJ.CJ315', 0),
 (u'cstu.CS.CS295', 0),
 (u'cstu.PY.PY228', 0),
 (u'cstu.CS.CS105', 0),
 (u'cstu.CS.CS314', 0),
 (u'cstu.CS.CS213', 0),
 (u'cstu.TU.T